In this ipynb the goal is to generate a synthetic data of demand and supply in inventories for a test environment



In [1]:
import pandas as pd
import random
from itertools import product
from dateutil.relativedelta import relativedelta 

In [2]:
# --- Setup ---
# Load products table
try:
    df_products = pd.read_csv("products_table.csv")
except FileNotFoundError:
    print("Error: 'products_table.csv' not found. Please make sure the file is in the same directory.")
    exit()

product_ids = df_products["ProductID"].unique().tolist()
unique_warehouses = df_products["WarehouseLocation"].unique().tolist()

In [3]:
# Create a fast lookup dictionary for products
product_lookup = df_products.set_index('ProductID').to_dict('index')

def random_date(start, end):
    """Generate random date between start and end"""
    return start + pd.to_timedelta(random.randint(0, (end - start).days), unit='d')

def generate_inventory_record(inv_id, product_id, warehouse, snapshot_date):
    """Generate one inventory record with injected issues."""
    product_row = product_lookup[product_id]
    
    # Normal values
    units_demanded = random.randint(50, 500)
    units_available = max(0, units_demanded - random.randint(0, 500))
    
    # --- DATA QUALITY INJECTIONS ---
    # Inject nulls (5% chance each)
    if random.random() < 0.05:
        units_demanded = None
    if random.random() < 0.05:
        units_available = None
    
    # Inject outliers (2% chance each, only if not None)
    if units_demanded is not None and random.random() < 0.02:
        units_demanded *= 100
    if units_available is not None and random.random() < 0.02:
        units_available *= 50
    
    return {
        "SnapshotInvID": inv_id,
        "ProductID": product_id,
        "ProductName": product_row["ProductName"],
        "WarehouseLocation": warehouse,
        "unitsAvailable": units_available,
        "unitsDemanded": units_demanded,
        "SnapshotDate": snapshot_date
    }

In [4]:
def generate_inventory_dataset(num_rows, filename):
    """Generate a large, realistic inventory dataset with time dimension and data issues."""
    
    # 1. DEFINE TIME DIMENSION
    start_date = pd.Timestamp("2022-01-01")
    end_date = pd.Timestamp("2024-09-01")
    
    # Create a list of monthly timestamps to use as our time dimension
    months = []
    current = start_date.replace(day=1) # Ensure we start on the 1st of the month
    while current <= end_date:
        months.append(current)
        current = current + relativedelta(months=1)
    
    # 2. CALCULATE AND SAMPLE UNIQUE COMBINATIONS
    # All possible combinations are (product x warehouse x month)
    all_combinations = list(product(product_ids, unique_warehouses, months))
    max_possible_rows = len(all_combinations)

    if num_rows > max_possible_rows:
        print(f"Warning: Requested {num_rows} rows, but only {max_possible_rows} unique combinations are possible. Capping at {max_possible_rows}.")
        num_rows = max_possible_rows

    # Randomly sample the desired number of unique combinations
    random.shuffle(all_combinations)
    selected_combinations = all_combinations[:num_rows]

    # 3. GENERATE THE RECORDS
    inventory_records = [
        generate_inventory_record(i + 1, prod_id, wh, snapshot_date) 
        for i, (prod_id, wh, snapshot_date) in enumerate(selected_combinations)
    ]
    
    df_inventory = pd.DataFrame(inventory_records)

    # 4. INJECT DUPLICATES
    # Inject duplicates (2% of the original unique dataset)
    num_duplicates = int(0.02 * len(df_inventory))
    if num_duplicates > 0:
        duplicates = df_inventory.sample(n=num_duplicates, replace=True)
        df_inventory = pd.concat([df_inventory, duplicates], ignore_index=True)

    # 5. FINALIZE AND SAVE
    # Reset the index
    df_inventory = df_inventory.reset_index(drop=True)
    
    # --- CORRECTION ---
    # Overwrite the existing 'SnapshotInvID' column with a new, unique sequence.
    # This removes the duplicate IDs and prevents creating a second 'InvID' column.
    df_inventory['SnapshotInvID'] = range(1, len(df_inventory) + 1)

    # Save dataset
    df_inventory.to_csv(filename, index=False)
    print(f"Generated {len(df_inventory)} total rows (with duplicates) to {filename}")

def generate_inventory_dataset(num_rows, filename):
    """Generate a large, realistic inventory dataset with time dimension and data issues."""
    
    # 1. DEFINE TIME DIMENSION
    start_date = pd.Timestamp("2022-01-01")
    end_date = pd.Timestamp("2024-09-01")
    
    # Create a list of monthly timestamps to use as our time dimension
    months = []
    current = start_date.replace(day=1) # Ensure we start on the 1st of the month
    while current <= end_date:
        months.append(current)
        current = current + relativedelta(months=1)
    
    # 2. CALCULATE AND SAMPLE UNIQUE COMBINATIONS
    # All possible combinations are (product x warehouse x month)
    all_combinations = list(product(product_ids, unique_warehouses, months))
    max_possible_rows = len(all_combinations)

    if num_rows > max_possible_rows:
        print(f"Warning: Requested {num_rows} rows, but only {max_possible_rows} unique combinations are possible. Capping at {max_possible_rows}.")
        num_rows = max_possible_rows

    # Randomly sample the desired number of unique combinations
    random.shuffle(all_combinations)
    selected_combinations = all_combinations[:num_rows]

    # 3. GENERATE THE RECORDS
    inventory_records = [
        generate_inventory_record(i + 1, prod_id, wh, snapshot_date) 
        for i, (prod_id, wh, snapshot_date) in enumerate(selected_combinations)
    ]
    
    df_inventory = pd.DataFrame(inventory_records)

    # 4. INJECT DUPLICATES
    # Inject duplicates (2% of the original unique dataset)
    num_duplicates = int(0.02 * len(df_inventory))
    if num_duplicates > 0:
        duplicates = df_inventory.sample(n=num_duplicates, replace=True)
        df_inventory = pd.concat([df_inventory, duplicates], ignore_index=True)

    # 5. FINALIZE AND SAVE
    # Reset and regenerate unique IDs after adding duplicates
    df_inventory = df_inventory.reset_index(drop=True)
    df_inventory['InvID'] = range(1, len(df_inventory) + 1)

    # Save dataset
    df_inventory.to_csv(filename, index=False)
    print(f"Generated {len(df_inventory)} total rows (with duplicates) to {filename}")


In [5]:
# --- EXECUTION ---
# Generate test dataset (5,000 unique rows + ~100 duplicates)
generate_inventory_dataset(5000, "inventory_test_with_issues.csv")

# Generate production dataset (50,000 unique rows + ~1,000 duplicates)
generate_inventory_dataset(50000, "inventory_production_with_issues.csv")

Generated 5100 total rows (with duplicates) to inventory_test_with_issues.csv
Generated 51000 total rows (with duplicates) to inventory_production_with_issues.csv
